In [ ]:
# | default_exp train
# | default_cls_lvl 3


In [ ]:
#| hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# train

> TODO

In [ ]:
# | export
import argparse, os
from pathlib import Path
import warnings

import pandas as pd
from transformers import logging

from kaggle_comp import config, train_dispatcher, utils
from kaggle_comp.config import CFG

# silence all the HF warnings
warnings.simplefilter("ignore")
logging.set_verbosity_error()
os.environ["TOKENIZERS_PARALLELISM"] = "false"


Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [ ]:
# | hide_input
import pdb
from fastcore.test import *


## Train a fold

In [ ]:
# | export
def run_fold(
    CFG,
    n_fold: int,
    experiment_name: str,
    run_id: str,
    grid_id: str,
    train_data=config.TRAINING_FILE,
    verbose: bool = True,
):

    train_config = train_dispatcher.experiments[experiment_name]
    model_name = f"{experiment_name}_{run_id}_{grid_id}_fold_{n_fold}"

    comp_trainer_cls = train_config["comp_trainer_cls"]
    comp_trainer = comp_trainer_cls(
        train_config=train_config, model_name=model_name, model_output_path=config.MODEL_OUTPUT, log_output_path=config.LOG_OUTPUT
    )

    if isinstance(train_data, pd.DataFrame):
        train_df = train_data.copy()
    else:
        train_df = pd.read_csv(train_data)

    train_df["is_valid"] = train_df["k_fold"] == n_fold

    log_df, oof_df = comp_trainer.train(
        CFG,
        train_df,
        n_fold=n_fold,
        run_id=run_id,
        grid_id=grid_id,
        experiment_name=experiment_name,
        verbose=verbose,
    )

    oof_preds = oof_df[[col for col in oof_df.columns if col.startswith("pred")]].values
    oof_targs = oof_df[[col for col in oof_df.columns if col.startswith("targ")]].values

    score = utils.comp_metric_score(oof_preds, oof_targs)

    if verbose:
        print("--- logging results ---")

    log_df.to_csv(Path(config.LOG_OUTPUT) / str(model_name + ".csv"), index=False)
    oof_df.to_csv(Path(config.LOG_OUTPUT) / str("oof_" + model_name + ".csv"), index=False)

    if verbose:
        print(f"Fold={n_fold}, Score={score}")


In [ ]:
# | export
if __name__ == "__main__" and utils.run_env != "local_nb":
    # instantiate argparser
    parser = argparse.ArgumentParser()

    # define args
    parser.add_argument("--experiment_name", type=str)
    parser.add_argument("--n_fold", type=int, default=0)
    parser.add_argument("--run_id", type=str, default="dummy")
    parser.add_argument("--grid_id", type=int, default=0)
    parser.add_argument("--verbose", type=str, default=True)

    # read in args from terminal
    args = parser.parse_args()

    run_fold(
        CFG=CFG,
        n_fold=args.n_fold,
        experiment_name=args.experiment_name,
        run_id=args.run_id,
        grid_id=args.grid_id,
        train_data=config.TRAINING_FILE,
        verbose=args.verbose,
    )


In [ ]:
class CFG:
    subset = 1.0
    n_fold = 5
    fold = [0, 1]
    random_seed = 42
    preprocess = "basic"
    postprocess = "none"
    augment = "none"
    train_folds = "train_folds.csv"
    experiments = ["blurr_deberta_v3_small"]
    grid_params = ["experiments", "fold"]

    hf_config_kwargs = {
       "num_labels": 6,
        "cls_dropout": 0.1,  # default = hidden_dropout_prob
        "pooler_dropout": 0.00,  # default = 0.0
        "hidden_dropout_prob": 0.1,  # default = 0.1
        "attention_probs_dropout_prob": 0.1,  # default = 0.1
        "layer_norm_eps": 1e-7 # default = 1e-7
    }

    batch_size = 8
    use_fp16 = True
    adam_beta1 = 0.9
    adam_beta2 = 0.999
    adam_eps = 1e-5
    weight_decay = 0.01
    max_grad_norm = None  # default = 1.0,

    n_unfrozen_epochs = 4
    unfrozen_lrs = [1e-6, 5e-4] #default = [3e-6, 5e-5]


In [ ]:
# manual debugging check
# run_fold(CFG=CFG, n_fold = 0, experiment_name = "blurr_deberta_v3_small", run_id = "2022_01_01_01_01", grid_id = 0)
run_fold(CFG=CFG, n_fold=0, experiment_name="blurr_deberta_v3_large", run_id="2022_01_01_01_01", grid_id=0)


Experiment: blurr_deberta_v3_large
Run: 2022_01_01_01_01 | Grid ID: 0 | Fold: 0
Training config: f{'comp_trainer_cls': <class 'kaggle_comp.framework_utils.fastai.FastaiCompTrainer'>, 'model_checkpoint': 'microsoft/deberta-v3-large', 'hf_config_kwargs': {'num_labels': 6, 'cls_dropout': 0.15, 'pooler_dropout': 0.0, 'hidden_dropout_prob': 0.1, 'attention_probs_dropout_prob': 0.1, 'layer_norm_eps': 1e-07}, 'hf_tokenizer_kwargs': {}, 'anchor_col': 'anchor', 'target_col': 'target', 'max_length': 512, 'include_labels': True, 'tok_kwargs': {}, 'batch_size': 8, 'adam_beta1': 0.9, 'adam_beta2': 0.999, 'adam_eps': 1e-06, 'weight_decay': 0.01, 'max_grad_norm': 1.0, 'save_best_model': True, 'use_fp16': True, 'n_frozen_epochs': 0, 'frozen_lr': None, 'n_unfrozen_epochs': 10, 'unfrozen_lrs': [1e-06, 2e-05]}
Building HF objects ...
Building DataLoaders ...
b0 shape: torch.Size([8, 512]); b1 shape: torch.Size([8, 6])
Building Learner ...


epoch,train_loss,valid_loss,mcrmse,time
0,0.522924,0.493651,0.507573,03:22
1,0.465757,0.474697,0.488732,03:22
2,0.434957,0.456236,0.469612,03:22
3,0.409501,0.479040,0.493536,03:22


Better model found at epoch 0 with mcrmse value: 0.5075727105140686.
Better model found at epoch 1 with mcrmse value: 0.4887324869632721.
Better model found at epoch 2 with mcrmse value: 0.46961212158203125.


--- saving model ---
--- logging results ---
Fold=0, Score=0.46961212158203125


## Export -

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()
